In [229]:
from sklearn.datasets import load_iris
import numpy as np
dataset = load_iris()
X = dataset.data
y = dataset.target
print(dataset.DESCR)


Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

In [230]:
attribute_means = X.mean(axis=0)
X_d = np.array(X >= attribute_means, dtype='int')


In [231]:
from collections import defaultdict
from operator import itemgetter

In [232]:
def train_feature_value(X, y_true, feature_index, value):
    #参数分别是数据集、类别数组、选好的特征索引值、特征值
    class_counts = defaultdict(int)
    for sample, y in zip(X, y_true):
        if sample[feature_index] == value:
            class_counts[y] += 1
    sorted_class_counts = sorted(class_counts.items(),key=itemgetter(1), reverse=True)
    most_frequent_class = sorted_class_counts[0][0]
    incorrect_predictions = [class_count for class_value, class_count
    in class_counts.items()
    if class_value != most_frequent_class]
    error = sum(incorrect_predictions)
    return most_frequent_class, error

In [233]:
def train_on_feature(X, y_true, feature_index):
    values = set(X[:,feature_index])
    predictors = {}
    errors = []
    for current_value in values:
        most_frequent_class, error = train_feature_value(X,y_true, feature_index, current_value)
        predictors[current_value] = most_frequent_class
        errors.append(error)
    total_error = sum(errors)
    return predictors, total_error

In [234]:
from sklearn.cross_validation import train_test_split
Xd_train, Xd_test, y_train, y_test = train_test_split(X_d, y, random_state=14)

In [235]:
all_predictors = {}
errors = {}

for feature_index in range(Xd_train.shape[1]):
    predictors, total_error = train_on_feature(Xd_train, y_train,feature_index)
    all_predictors[feature_index] = predictors
    errors[feature_index] = total_error
print(all_predictors)
print(errors)
print(sorted(errors.items(), key=itemgetter(1)))
best_feature, best_error = sorted(errors.items(), key=itemgetter(1))[0]
print(best_feature)
print(best_error)

{0: {0: 0, 1: 2}, 1: {0: 1, 1: 0}, 2: {0: 0, 1: 2}, 3: {0: 0, 1: 2}}
{0: 41, 1: 58, 2: 37, 3: 37}
[(2, 37), (3, 37), (0, 41), (1, 58)]
2
37


In [236]:
model = {'feature': best_feature,'predictor': all_predictors[best_feature]}


In [237]:
print(model)

{'feature': 2, 'predictor': {0: 0, 1: 2}}


In [238]:
def predict(X_test, model):
    feature = model['feature'] 
    predictor = model['predictor']
    y_predicted = [predictor[int(sample[feature])] for sample in X_test]
    return y_predicted

In [239]:
y_predicted = predict(Xd_test, model)
accuracy = np.mean(y_predicted == y_test) * 100
print("The test accuracy is {:.1f}%".format(accuracy))

The test accuracy is 65.8%
